In [1]:
import sys
import argparse
import os
import torch
from collections import OrderedDict
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import seaborn as sns
import pylab as py
import time
from pyDOE import lhs
import warnings
sys.path.insert(0, '../../../Scripts/')
from models_imperfect import PGNN
from apinn import *
# from ../Scripts/helper import *

warnings.filterwarnings('ignore')

np.random.seed(1234)

In [2]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda:3')
else:
    device = torch.device('cpu')

## Hyper-parameters

In [3]:
num_epochs = 10000
lambda_mse = 1
tr_frac = 0.4

#architecture for the models
net_hid_dim = 50
net_num_layer = 4

### Load Data

In [4]:
data = np.loadtxt('../../../datasets/tossing_trajectories.txt')
x = data[:, 0:6]
labels = data[:, 6:]

# training and test split
n_obs = int(tr_frac * x.shape[0])
train_x , train_y = x[:n_obs,:] , labels[:n_obs, :] 
test_x , test_y = x[n_obs:,:] , labels[n_obs:, :]

data_dim = train_x.shape[1]
out_dim = labels.shape[1]

#### Models

In [5]:
net = PGNN(in_dim = data_dim, out_dim = out_dim, hid_dim=net_hid_dim, num_layers=net_num_layer).to(device)

In [6]:
APINN = Tossing_APINN(train_x, train_y, test_x, test_y, net, device, num_epochs, lambda_mse)

### Training

In [7]:
APINN.train()

[Epoch 0/10000] [MSE loss: 0.996759] [Phy loss: 110.899002] [Total loss: 273.497559] [Lambda mse : 31.392628]
[Epoch 100/10000] [MSE loss: 0.081032] [Phy loss: 10.001344] [Total loss: 11.732128] [Lambda mse : 13.531187]
[Epoch 200/10000] [MSE loss: 0.063477] [Phy loss: 7.166821] [Total loss: 10.235534] [Lambda mse : 41.732868]
[Epoch 300/10000] [MSE loss: 0.060130] [Phy loss: 5.679905] [Total loss: 8.322547] [Lambda mse : 20.258808]
[Epoch 400/10000] [MSE loss: 0.053667] [Phy loss: 5.642215] [Total loss: 8.071356] [Lambda mse : 21.665174]
[Epoch 500/10000] [MSE loss: 0.092520] [Phy loss: 5.058714] [Total loss: 7.626081] [Lambda mse : 20.384583]
[Epoch 600/10000] [MSE loss: 0.048251] [Phy loss: 5.072522] [Total loss: 6.967111] [Lambda mse : 20.813732]
[Epoch 700/10000] [MSE loss: 0.048751] [Phy loss: 4.611448] [Total loss: 6.481643] [Lambda mse : 26.241053]
[Epoch 800/10000] [MSE loss: 0.080445] [Phy loss: 4.711361] [Total loss: 6.772224] [Lambda mse : 14.458344]
[Epoch 900/10000] [MSE 

In [8]:
nsamples = 500
train_mean_y, train_std_y = APINN.uncertainity_estimate(APINN.train_x, nsamples, [APINN.Ymean, APINN.Ystd])

train_predictions = torch.Tensor(train_mean_y).float().to(device)

std_y_t = torch.Tensor(APINN.Ystd).float().to(device)
mean_y_t = torch.Tensor(APINN.Ymean).float().to(device)
std_x_t = torch.Tensor(APINN.Xstd).float().to(device)
mean_x_t = torch.Tensor(APINN.Xmean).float().to(device)

train_y = (APINN.train_y * std_y_t) + mean_y_t
train_x = (APINN.train_x * std_x_t) + mean_x_t

mse_train = ((train_predictions - train_y)**2).mean()
phy_train = torch.mean(APINN.physics_loss(train_x, train_predictions, [0, 1], [0, 1])).detach().cpu().numpy()

print("MSE on Training Set : ", mse_train.detach().cpu().numpy())
print("Physical Consistency: ", phy_train)

MSE on Training Set :  0.42464492
Physical Consistency:  0.64041924


In [9]:
test_mean_y, test_std_y = APINN.uncertainity_estimate(APINN.test_x, nsamples, [APINN.Ymean, APINN.Ystd])

test_predictions = torch.Tensor(test_mean_y).float().to(device)

test_y = (APINN.test_y * std_y_t) + mean_y_t
test_x = (APINN.test_x * std_x_t) + mean_x_t

mse_test = ((test_predictions - test_y)**2).mean()
phy_test = torch.mean(APINN.physics_loss(test_x, test_predictions, [0, 1], [0, 1])).detach().cpu().numpy()

print("MSE on Test Set : ", mse_test.detach().cpu().numpy())
print("Physical Consistency: ", phy_test)

MSE on Test Set :  0.61379516
Physical Consistency:  0.7096468
